In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [ ]:
#Used pandas to read csv file
df = pd.read_csv('damped_harmonic_oscillator_data.csv')

t = df['t'].to_numpy()
x = df['x'].to_numpy()
print(df)
plt.plot(t,x)
plt.xlabel('Time')
plt.ylabel('Position')
plt.title('Given data')
plt.show()

In [ ]:
T = t.reshape(-1, 1)  # time
X = x.reshape(-1, 1)  # position
print(T[0],X[0])
print(T.dtype)

In [ ]:
#splitting data [40 % testing and 60% training]
#20%(test) ---- 60% (train)--- 20% (test)

marker_1 = int(0.2 * len(df))
marker_2 = int(0.8 * len(df))

# TRAINING DATA
train_data = df.iloc[marker_1:marker_2]
T_train = train_data.iloc[:, :1].to_numpy().reshape(-1, 1)  # time
X_train = train_data.iloc[:, 1:].to_numpy().reshape(-1, 1)  # position

# TESTING DATA
first_half_test = df.iloc[:marker_1]
second_half_test = df.iloc[marker_2:]

test_data = pd.concat([first_half_test, second_half_test])

T_test = test_data.iloc[:, :1].to_numpy().reshape(-1, 1)  # time
X_test = test_data.iloc[:, 1:].to_numpy().reshape(-1, 1)  # position

# Verify shapes
print("Shape of T_train:", T_train.shape)
print("Shape of X_train:", X_train.shape)
print("Shape of T_test:", T_test.shape)
print("Shape of X_test:", X_test.shape)

In [ ]:
# Convert numpy arrays to PyTorch tensors
T_train_tensor = torch.tensor(T_train, dtype=torch.float32)
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
T_test_tensor = torch.tensor(T_test, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)

In [ ]:
torch.seed()

class SGDRegression(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(1, 4)  # Input layer [ 1 input and 1 output]
        self.fc2 = nn.Linear(4, 4)  # hidden layer 1
        self.fc3 = nn.Linear(4, 1)  # hidden layer 2
    
    def forward(self, x):
        x = F.relu(self.fc1(x)) #Activation functions
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

model = SGDRegression()
loss_fn = torch.nn.MSELoss() # we want to model mean square loss
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
EPOCHS = 50

In [ ]:
for name, param in model.named_parameters():
    print(name, param)


In [ ]:
losses = []
pred = 0
for epoch in range(EPOCHS):
    ### PASS DATA THROUGH MODEL ###
    X_pred = model(T_train_tensor) 
    loss = loss_fn(X_pred, X_train_tensor)
    losses.append(loss.item())
    pred = X_pred
    ### OPTIMIZE MODEL : The gradient loss thing ###
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()


In [ ]:
plt.plot(range(EPOCHS), losses)
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss')
plt.show()

In [ ]:
for name, param in model.named_parameters():
    print(name, param)

In [ ]:
#Evaluating for future and past
model.eval()
with torch.no_grad():
    X_test_pred = model(T_test_tensor)
X_test_pred = X_test_pred.numpy()

# Plot actual vs predicted values
plt.plot(T_test, X_test, 'b.', label='Actual')
plt.plot(T_test, X_test_pred,'r.', label='Predicted')
plt.xlabel('Time (t)')
plt.ylabel('Position (x)')
plt.legend()
plt.title('Actual vs Predicted Values on Testing Data')
plt.legend()
plt.show()

In [ ]:
model.eval()
with torch.no_grad():
    X_train_pred = model(T_train_tensor)
X_train_pred = X_train_pred.numpy()

# Plot actual vs predicted values
plt.plot(T_train, X_train, 'b.', label='Actual')
plt.plot(T_train, X_train_pred,'r.', label='Calculated')
plt.xlabel('Time (t)')
plt.ylabel('Position (x)')
plt.legend()
plt.title('Actual vs Calculated Values on Training Data')
plt.legend()
plt.show()